In [1]:
import time
import copy
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt

import sklearn
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

from models import *
from utils import *
from datasets import *

d:\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# MM-II

In [4]:
model = AttnFusion(visual_size=100, metric_size=9)
visual = torch.ones((1,100))
metric = torch.ones((1,9))

feats = model(user_preference=metric, visual_feature=visual)
feats.shape

torch.Size([100])

# MMModel

In [2]:
model = MMModel(MM_I_wPATH="./results/encoder_weights.pt", MM_II_wPATH=None, freeze=(False, False), device=torch.device('cpu'))

param1 = sum(p.numel() for p in model.MM_I.parameters() if p.requires_grad)
param2 = sum(p.numel() for p in model.MM_II.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters() if p.requires_grad)

param1, param2, total

(2697986, 11289915, 13987901)

In [3]:
sum(p.numel() for p in model.MM_II.cnn.parameters() if p.requires_grad)

11224164

In [4]:
resnet = ResNet(BasicBlock, [1, 1, 1, 1])
sum(p.numel() for p in resnet.parameters() if p.requires_grad)

4953444

In [5]:
img = torch.ones((1,10,1000,1000))
out = resnet(img)
out.shape

torch.Size([1, 100])

# demo

In [2]:
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Data Preprocessing

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trainTransform  = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])
trainset = tv.datasets.MNIST(root='./data',  train=True,download=True, transform=transform)
testset = tv.datasets.MNIST(root='./data',  train=False,download=True, transform=transform)

In [3]:
traindata = [i[0] for i in trainset]
trainlabel = [i[1] for i in trainset]
testdata = [i[0] for i in testset]
testlabel = [i[1] for i in testset]

X = traindata + testdata
y = trainlabel + testlabel

In [4]:
X = torch.vstack(X)
X.shape

torch.Size([70000, 28, 28])

In [5]:
X = X.unsqueeze(1)
X.shape

torch.Size([70000, 1, 28, 28])

In [6]:
# z, answers = model()

z = np.load("./tsne_data_reducted_normalised.npy")
z = torch.tensor(z)
z.shape

torch.Size([70000, 3])

In [7]:
def get_Ihat(Z):
    mat = np.zeros((1000,1000))
    for i in range(len(Z)):
        xx = min(int(np.floor(Z[i,0]*1000)), 999)
        yy = min(int(np.floor(Z[i,1]*1000)), 999)
        mat[xx,yy] = Z[i, 2]
    return mat

# VI = VisualImitation(device='cpu')
# I_hat = VI(z[:128,:])
I_hat = get_Ihat(Z=z)
I_hat = torch.tensor(I_hat).unsqueeze(0)
I_hat.shape

torch.Size([1, 1000, 1000])

In [8]:
model = HumanModel(cnn_layers=[2,2,2,2], metric_num=9, hidden_dim=100)
q1, q2, q3 = model(I_hat=torch.ones((1,10,1000,1000)), z=z[:1024,:2], labels=None, x=None)

./metrics/get_scag.r


In [11]:
q1, q2, q3

(tensor([[0.4802]], grad_fn=<SigmoidBackward0>),
 tensor([[0.4878]], grad_fn=<SigmoidBackward0>),
 tensor([[0.4832]], grad_fn=<SigmoidBackward0>))

In [ ]:
# I_hat = model.VI(z)
I_hat = torch.ones((1,10,1000,1000))
answers = model.MM_II(I_hat=I_hat, z=z, labels=y, x=X)

out = model.MM_II(z)